In [22]:
import sys
sys.path.insert(1, '../src')
import condensate_analysis as ca
import matplotlib.pyplot as plt

In [ ]:
from condensate_analysis import read

In [44]:
# Import: Image file -> numpy array
def read_image(file):
"""Read image into numpy ndarray.

Should handle nd2, czi, tif, and other common file type.
Handling is based on file extention only.
Dimension order is set with the convention: Channel, Slice, Row, Col.
"""
    if file.endswith(".czi"):
        img = czifile.imread(file).squeeze() # remove dummy dimensions
    elif file.endswith(".nd2"):
        img = nd2.imread(file)
        img = img.swapaxes(0,1) # places channel before slice
    else:
        try:
            img = skimage.io.imread(file)
            img = img.swapaxes(0,1) # I think this is also necessary here, but not sure. 
            # Consider checking for larger value between slice and channel for placement.
        except OSError:
            print("Invalid file extention")
    return img

# Mask: numpy array -> blob mask
    # should accept 2D or 3D
    # segmentaion methods: Otzu threshold,
    # dilation or constriction option
    # watershed option

# Featurize: array, mask, feature_list -> features table
    # X, Y, Area, MaxIntensity, SumIntensity, Circulatiry


In [7]:
img = ca.read.read_image("../testdata/test.nd2")
print(img.shape)
img = ca.read.read_image("../testdata/test.czi")
print(img.shape)

(2, 21, 2048, 2048)
(2, 15, 512, 512)


In [51]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max

def mask_image(img, threshold_method="otzu", watershed=False):
    if threshold_method == "otzu":
        thresh = threshold_otsu(img)
    binary = img > thresh
    no_edge_binary = clear_border(binary)
    label_image = label(no_edge_binary)
    if watershed == True:
        distance = ndi.distance_transform_edt(binary)
        distance = ndi.gaussian_filter(distance,5)
        coords = peak_local_max(distance, footprint=np.ones((25, 25)), labels=binary)
        mask = np.zeros(distance.shape, dtype=bool)
        mask[tuple(coords.T)] = True
        markers, _ = ndi.label(mask)
        labels = watershed(-distance, markers, mask=binary)
        mask = label_image
    elif watershed == False:
        mask = label_image
    else: 
        TypeError
        print("watershed accepts True or False values only")
    return mask

In [53]:
mask = mask_image(img[0,0])
plt.imshow(mask[7,:,:])

TypeError: 'bool' object is not callable

In [54]:
img[0,0]

array([[2, 0, 0, ..., 2, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 0, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 3, 1, 1],
       [1, 0, 1, ..., 1, 1, 1]], dtype=uint8)